In [2]:
import pandas.util.testing as tm
from tensorflow.keras.models import load_model
from flask import Flask,request, jsonify
import os
import numpy as np
import uuid
app = Flask(__name__)


import warnings
warnings.filterwarnings('ignore')

### 1. Set expectations

In [3]:
Expected = {
"usd_pledged_real":{"min":0.0,"max":20338986.27},
"backers":{"min":0,"max":219382},
"usd_goal_real":{"min":0.01,"max":166361390.71}
}

### 2. Import the Model

In [4]:
# Load Model for ANN 
model = load_model(os.path.join(os.getcwd(),"enpowered_model.h5"))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 24        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 199
Trainable params: 199
Non-trainable params: 0
__________________________________________________________

### 3. Set parameter conditions

In [6]:
@app.route('/api', methods=['POST'])  # Define the decorator and create application and use request to send data
def project_prediction():
    content = request.json
    errors = []
    
    # Check Min and Max and check Names
    for name in content:
        if name in Expected:
            expected_min = Expected[name]['min']
            expected_max = Expected[name]['max']
            value = content[name]
            if value < expected_min or value > expected_max:
                errors.append(f"Out of bounds: {name}, has value of: {value}, but it should be between {expected_min} and {expected_max}.")
        else:
            errors.append(f"Unexpected field: {name}")
    
    # Check values of the existing names and thier values
    for name in Expected:
        if name not in content:
            errors.append(f"Missing_value:{name}")
    if len(errors)<1:  # Error list is empty
        x = np.zeros((1,3))     
        x[0,0]  = content['usd_pledged_real']
        x[0,1]  = content['backers']
        x[0,2]  = content['usd_goal_real']

            
    # Prediction
        prediction = model.predict(x)
        Project_Result = float(prediction[0])
        response = {"id": str(uuid.uuid4()), "Project_Result": Project_Result, "errrors": errors}
        
    else:
        response = {"id": str(uuid.uuid4()), "errrors": errors}
        
    return jsonify(response)

   # Run the application
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
                

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
